# Reaching multiple targets with a manipulator
The objective of this exercice is to implement a trajectory with a manipulator to reach multiple targets.

We provide a basic example for reaching one point, and you have to modify it to reach multiple points in a sequence. Below is the basic example. Next we explain it and we guide you to the final result.


In [3]:
# %load arm_example.py
from crocoddyl import *
import pinocchio
import numpy as np

robot = loadTalosArm()
robot.initDisplay(loadModel=True)

robot.q0.flat[:] = [  2,1.5,-2,0,0,0,0 ]
robot.model.armature[:] = .2
frameId = robot.model.getFrameId('gripper_left_joint')
DT = 1e-2
T  = 25

target = np.array([ 0.4,0  ,.4 ])

robot.viewer.gui.addSphere('world/point',.1,[1,0,0,1])  # radius = .1, RGBA=1001
robot.viewer.gui.applyConfiguration('world/point', target.tolist()+[0,0,0,1] )  # xyz+quaternion
robot.viewer.gui.refresh()

# Create the cost model
costTrack = CostModelFrameTranslation(robot.model,frame=frameId,ref=target)
costXReg  = CostModelState(robot.model,StatePinocchio(robot.model))
costUReg  = CostModelControl(robot.model)

# Create the action model with empty costs
model     = DifferentialActionModel(robot.model)
termmodel = DifferentialActionModel(robot.model)

# Then let's added the running and terminal cost functions
model.costs.addCost( name="pos", weight = 1, cost = costTrack)
model.costs.addCost( name="xreg", weight = 1e-4, cost = costXReg)
model.costs.addCost( name="ureg", weight = 1e-7, cost = costUReg)

termmodel.costs.addCost( name="pos", weight = 1000, cost = costTrack)
termmodel.costs.addCost( name="xreg", weight = 1e-4, cost = costXReg)
termmodel.costs.addCost( name="ureg", weight = 1e-7, cost = costUReg)

# Create the problem
x0 = np.concatenate([ m2a(robot.q0), np.zeros(robot.model.nv)])
problem = ShootingProblem(x0, [ IntegratedActionModelEuler(model) ]*T,
                          IntegratedActionModelEuler(termmodel))

# Creating the DDP solver for this OC problem, defining a logger
ddp = SolverDDP(problem)
ddp.callback = [CallbackDDPVerbose() ]

# Solving it with the DDP algorithm
ddp.solve()

# Visualizing the solution in gepetto-viewer
for x in ddp.xs: robot.display(a2m(x))

print('Finally reached = ',
      ddp.datas()[T].differential.costs['pos'].pinocchio.oMf[frameId].translation.T)


iter 	 cost 	      theta 	    gamma 	  muV 	      muLM 	 alpha
   0  4.03759e+02  4.63479e-05  3.91802e+02  1.00000e-09  1.00000e-09  1.0000
   1  2.63286e+02  7.37335e-05  6.42758e+02  1.00000e-09  1.00000e-09  0.2500
   2  2.20577e+02  4.50380e-05  3.86323e+02  1.00000e-09  1.00000e-09  1.0000
   3  1.18659e+02  4.71622e-05  3.86564e+02  1.00000e-09  1.00000e-09  1.0000
   4  1.05175e+02  1.38233e-05  1.20665e+02  1.00000e-09  1.00000e-09  1.0000
   5  7.09226e+01  2.10900e-05  1.79137e+02  1.00000e-09  1.00000e-09  1.0000
   6  3.00234e+01  1.56809e-05  1.35583e+02  1.00000e-09  1.00000e-09  1.0000
   7  1.12396e+01  5.11524e-06  4.39839e+01  1.00000e-09  1.00000e-09  1.0000
   8  1.05845e+01  2.77501e-07  2.34366e+00  1.00000e-09  1.00000e-09  1.0000
   9  1.03916e+01  1.19610e-07  1.02553e+00  1.00000e-09  1.00000e-09  0.2500
iter 	 cost 	      theta 	    gamma 	  muV 	      muLM 	 alpha
  10  1.03757e+01  3.38563e-08  2.90998e-01  1.00000e-09  1.00000e-09  1.0000
  11  1.03267e+0

## DifferentialActionModel for Pinocchio ABA
This scenario uses an advanced action model, that compute 2nd order differential dynamics with Pinocchio, and can accept several cost models. The action model is then tailored for a robot application, but it is at the same time quite module, as you can modify the robot dynamics by changing Pinocchio model, and adjust the cost by summing a new term, tunning a term or creating your own cost term.

## Cost models

Let's start with the cost. A cost model can compute a scalar cost and the gradient and hessian of the cost. All the models implemented are indeed computing a cost residuals and computing the hessian with the Gauss approximation.

We implementing basic cost for controlling a frame placement, translation or velocity, controlling the center of mass position, and definining a cost in the state space and in the control space. In the example above, we used the CostModelFrameTranslation, implementing a 3d position task, and the two costs State and Control acting as regularizers.

As for any other models in Croccodyl, you have to create a data with the model to be able to use it. The cost data must be create from a pinocchio data (the rational is that the pinocchio data used to compute the dynamics should be re-used to compute the cost).


In [8]:
dataTrack = costTrack.createData(robot.data)

### Cost Frame Position

You define a frame ID and the reference position as a 3D array. The cost is the distance between the center of the frame and the target. The cost is only a function of x (and only of the q part of x). You can double check the 0 in its gradient.

In [12]:
pinocchio.updateFramePlacements(robot.model,robot.data)
pinocchio.computeJointJacobians(robot.model,robot.data,a2m(x))
costTrack.calcDiff(dataTrack,x0,None)
print(dataTrack.Lx,dataTrack.Lu)

(array([-8.72516471e-03,  5.88825598e-03, -2.05031219e-04, -9.92433215e-05,
        1.43041681e-03, -7.60960409e-05,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00]), 0)


### Cost State
Here you must define a State model, which defines what are the dimension of the state and its tangent, and what are the exponential/integrate and difference/log operators. This is done automatically from Pinocchio, but we added the State class in the API to make it explicit and generic.

The cost State uses a reference in state space (State.zero() by default). The cost is the distance, computed with state.difference between the current state and the reference. Hence the cost regularizes both position and velocity.

### Cost Control

Explain the cost control here


## Add cost models to the Differential action model
Explain here the addCost function

## Create the problem with integated cost
Recall quickly the integrated cost and refer to the cart-pole example.

## Callbacks

Explain quickly the two important callbacks: verbose and display. 

# Modifying the example

Start by defining several targets (let's say 4 targets, all at x=0.4, and at y and z being either 0 or 0.4), and display then in the viewer.


The shooting problem will be composed of 4 sequences of action models, each sequences with T shooting "running" nodes, and 1 terminal node. The running nodes mostly have regularization terms, while the terminal nodes have a strong cost toward the corresponding target.

[ R1,R1,R1 ... R1,T1, R2,R2 .... R2, T2, R3 ... R3, T3, R4 ... R4 ] , T4

First create 4 running models and 4 terminal models.

Then add the 3 costs pos, xreg and ureg to each running and terminal model. In the running models, the pos cost should be low, and it should be high in the terminal models.

Now create a shooting problem.

In [13]:
seq0 = [runningmodels[0]]*T + [terminalmodel[0]]
seq1 = [runningmodels[1]]*T + [terminalmodel[1]]
seq2 = [runningmodels[2]]*T + [terminalmodel[2]]
seq3 = [runningmodels[3]]*T 
problem = ShootingProblem(x0,seq0+seq1+seq2+seq3,terminalmodel[3])

NameError: name 'runningmodels' is not defined

Create a DDP solver for this problem and run it. 

In [15]:
ddp = SolverDDP(problem)
ddp.solve()

([array([ 2. ,  1.5, -2. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
          0. ,  0. ,  0. ]),
  array([ 2.00276016e+00,  1.49536806e+00, -1.99874790e+00,  8.12648182e-06,
         -1.37732679e-03,  2.98750766e-04,  9.09930966e-05,  2.76015874e+00,
         -4.63193763e+00,  1.25210424e+00,  8.12648182e-03, -1.37732679e+00,
          2.98750766e-01,  9.09930966e-02]),
  array([ 2.00811977e+00,  1.48646633e+00, -1.99632191e+00,  2.81154769e-05,
         -3.99490322e-03,  8.81722840e-04,  2.54202696e-04,  5.35961319e+00,
         -8.90173520e+00,  2.42599041e+00,  1.99889950e-02, -2.61757643e+00,
          5.82972074e-01,  1.63209600e-01]),
  array([ 2.01594788e+00,  1.47363666e+00, -1.99276627e+00,  6.40730041e-05,
         -7.72775418e-03,  1.74149161e-03,  4.73982137e-04,  7.82810678e+00,
         -1.28296631e+01,  3.55563539e+00,  3.59575272e-02, -3.73285096e+00,
          8.59768771e-01,  2.19779441e-01]),
  array([ 2.02613700e+00,  1.45720129e+00, -1.98810083e+00,  1.20139

Well, it should not work. Not from the first shot at least: the DDP solver is likely not strong enough to accept the random weights that you have selected. 

If it is working nicely from the first shot, display it in the viewer and go take a coffee. But you will likely have to tweak the gains to make it work. It is advice to first optimize only sequence 1. When you are happy with it, add sequence 2 and optimize again, etc.


## Penalty
The solver works with double precisions, so it is quite robust to high weight. 10000 is likely to be accepted for example. But if you make the problem too difficult, the solver will break. 

In that case, you can implement a simple penalty solver by setting the weight to be 10**i, and creating a for loop to explore i from 0 to 5. At each iteration of the loop, run the solver from the previous solution and for few iterations only.

In [16]:
for i in range(1,6):
    terminalmodel[1].costs['pos'].weight = 10**i
    ddp.solve(init_xs=ddp.xs,init_us=ddp.us,maxiter=10)


NameError: name 'terminalmodel' is not defined